<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">Tipología y ciclo de vida de los datos · PRAC1</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Práctica 1

## Los elementos de la tabla periódica

Desarrollo del código en python.

In [230]:
#Cargamos todas las librerías necesarias
import requests
from bs4 import BeautifulSoup
import csv
import re
import pandas as pd



#Cargamos la web principal de donde extraeremos los principales atributos
url_page = 'http://www.rinconeducativo.org/contenidoextra/tablasperiodicas/tabla/tabla.php#tit'

#Accedemos a ella y mediante beautifoul soup iremos extrayendo lo que necesitemos
page = requests.get(url_page).text 
soup = BeautifulSoup(page, "lxml")

#---------------------------------------------------------------------------------------
#FUNCIONES GENERALES QUE UTILIZAREMOS A LO LARGO DE LA PRÁCTICA
#---------------------------------------------------------------------------------------
#Eliminación de las tildes
def tildes(lista):
    a1=[i.replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u").replace("ñ","n") for i in lista]
    return a1

#Función que nos muestra si todos los elementos son iguales
def todos_elementos_iguales(lista):
    longitud = len(lista)
    for i in range(longitud-1):
        if lista[i] != lista[i+1]:
                return False
    return True

#---------------------------------------------------------------------------------------
#Extracción del NÚMERO ATÓMICO, NOMBRE ABREVIADO Y NOMBRE COMPLETO
#---------------------------------------------------------------------------------------
#Extraemos la etiqueta h2 donde se encontrarán los atributos anteriormente citados
t1 = soup.find_all('h2')   
#Eliminamos la primera fila innecesaria
t1.pop(0)                                                                   

#Extraemos los valores para el numero atómico encontrando los elementos numéricos
n_atomico = [re.findall("\d{3}|\d{2}|\d{1}", d.text)[0] for d in t1]   
#Extremos el nombre abreviado y el nombre completo detectando los caracteres alfabéticos
icon = [re.findall("[A-Za-zóíú]+", d.text) for d in t1]                     


#Creamos listas vacías que rellenaremos
elemento=[]
nombre=[]

#Realizamos un bucle donde guardaremos por separado el nombre completo y el nombre abreviado
for i in range(len(icon)):                                                   
    elemento.append(icon[i][0])
    nombre.append(icon[i][1])

#Corregimos algunas incoherencias mostradas sobre los datos
nombre[nombre.index('Esta')]='Estaño'                                       
nombre[nombre.index('Ars')]='Arsénico'

#Eliminamos todas las tildes para una correcta lectura mediante la función anteriormente definida
nombre=tildes(nombre)

#---------------------------------------------------------------------------------------
#Extracción del PERIODO, GRUPO y MASA ATÓMICA
#---------------------------------------------------------------------------------------
#Extraemos la etiqueta h3 donde se encontrarán estos tres atributos.
t2 = soup.find_all('h3')
#Eliminamos la primera fila al resultar innecesaria
t2.pop(0)
#Extraemos el texto correxpondiente a esta etiqueta
texto2=[d.text for d in t2]

#Este parámetro nos guardará los número de uno en uno
l1= [re.findall("\d{1}", d.text) for d in t2] 
#Este parámetro nos guardará los números de dos cifras
l2= [re.findall("\d{2}", d.text) for d in t2]

#Construimos dos lista en blanco donde iremos introduciendo los valores
masa=[]
periodo_grupo=[]

#Realizamos un bucle donde introduciremos los valores brutos de la masa y del periodo/grupo en otro
for i in range(len(l1)//2):  
    #Los elementos del periodo/grupo se encuentran en las posiciones pares de la lista
    periodo_grupo.append(l1[2*i]) 
    #La masa se encuentra en las posiciones impares
    masa.append(texto2[2*i+1])
    
#Separamos el periodo del grupo
periodo=[]
grupo=[]
#Recorreremos todos los elementos de la lista periodo_grupo
for i in range(len(periodo_grupo)):
    #Periodo tienen todos los elementos y se corresponde con la primera cifra de la lista periodo_grupo 
    periodo.append(periodo_grupo[i][0])
    #Grupo no tendrán todos los elementos
    if len(periodo_grupo[i])==1: 
        grupo.append('-')                  #Si no tiene grupo se colocará un -
    if len(periodo_grupo[i])==2:
        grupo.append(l1[2*i][1])           #Si tiene una cifra se usará l1
    if len(periodo_grupo[i])==3:
        grupo.append(l2[2*i][0])           #Si tiene dos cifras se usará l2
        

#Con respecto a la masa definiremos una función para que nos extraiga los valores en umas.
#Esta función nos eliminará los parámetros de texto innecesarios y nos dejará exclusivamente la masa
def extraer_masa(lista):
    m1=[re.sub("Peso atómico:", "", i) for i in lista]                       
    m2=[re.sub("uma", "", i) for i in m1]                                    
    m3=[i.replace("\u200b","").replace('[','').replace(']*','')              
        .replace(']5',"").replace(" ", "").replace(".",",") for i in m2]
    m4=[i[0:5] for i in m3 ]
    return m4

#---------------------------------------------------------------------------------------
#Extracción de la CONFIGURACIÓN ELECTRÓNICA y el AÑO DE DESCUBRIMIENTO
#---------------------------------------------------------------------------------------

#De la etiqueta h4 extraemos la configuracion y descubrimiento
t3=soup.find_all('h4')
#Definimos el texto
texto3=[d.text for d in t3]

#Creamos las lista donde guardamos los elementos pares e impaes de la misma
#Los pares nos servirán para la configuración electrónica
par=[]
#Los impares para  la configuración electrónica
impar=[]

#Realizamos un bucle para guardar los valores
for i in range(len(texto3)//2):
    par.append(texto3[2*i])
    impar.append(texto3[2*i+1])

#Extraemos la configuración electrónica eliminando el texto innecesario
electronic = [re.sub("Configuración electrónica:","", d) for d in par]
 
#Definimos una función que nos da el año   
#Se cambia la forma de algunos años, se seleccionan 4 cifras para encontrar el año
# y además se busca donde se encuentra la palabra Prehistoria
#para posteriormente poderla introducir mediante un bucle
def año(lista):   
    discover  =  [re.sub("/69","-1969", d) for d in lista]      
    discover2 =  [re.sub("/70","-1970", d) for d in discover] 
    discover3 =  [re.findall("\d{4}", d) for d in discover2]
    discover4 =  [re.search("Prehistoria", d) for d in lista]
    for i in range(len(discover)):
        if len(discover3[i])==2:
            discover3[i]='-'.join(discover3[i])
        elif discover4[i] != None:
            discover3[i]='Prehistoria'
        else:
            discover3[i]=discover3[i][0]
    return discover3


#---------------------------------------------------------------------------------------
#Extracción de la ESTRUCTURA CRISTALINA del elemento
#---------------------------------------------------------------------------------------
#Conseguimos el texto de las etiquetas donde aparece
t4=soup.find_all('h5')

#Eliminamos el texto innecesario
divi = [re.sub("Estructura cristalina:", "", d.text) for d in t4]

#Creamos una lista donde guardaremos los datos
estructura=[]

#Realizaremos un bucle donde eliminaremos elementos innecesarios, los espacios del principio y el final, y 
#pondremos en mayúscula la primera letra
for j in range(len(divi)):
    estructura.append(divi[j].replace("\n"," ").replace("\t"," ").rstrip().lstrip().capitalize())
    
#Eliminamos las tiles que puedan aparecer
estructura=tildes(estructura)

#---------------------------------------------------------------------------------------
#Extracción de la DENSIDAD del elemento
#---------------------------------------------------------------------------------------
#Aunque en la página que estábamos tratando hasta ahora aparecían algunas densidades, gran parte de ellas no estaban presentes
#por lo que estas serán extraídas de una nueva página.

#Enlace a la nueva página de donde extraeremos las densidades.
#Esta página es general por lo que en primer lugar debemos obtener los enlaces que nos permiten acceder a cada elemento
#para poder entrar de esta forma a su densidad
url_page1 = 'https://es.wikipedia.org/wiki/Tabla_peri%C3%B3dica_de_los_elementos'
#Accedemos a ella
page1 = requests.get(url_page1, headers=headers).text 
soup1 = BeautifulSoup(page1, "lxml")
#Guardamos todos los elementos con la etiqueta "a" donde se encontrarán los enlaces
t5=soup1.find_all('a')

#Creamos una lista donde guardaremos todos los enlaces de la etiqueta a
urls=[]
#Realizamos un bucle donde extraemos el valor de 'href'
for i in t5:
    urls.append(i.get('href'))

#Eliminaremos aquellos elementos que no nos dirijan a los elementos de la tabla periódica
#De esta forma eliminaremos los elementos innecesarios por arriba y por abajo
urls=urls[urls.index('/wiki/Elementos_del_periodo_1'):urls.index('/wiki/Estado_de_agregaci%C3%B3n_de_la_materia')]

#Localizamos aquellos datos que no son necesarios inspeccionar para localizar los elementos
d1=[re.search('Elementos', i) for i in urls]
d2=[re.search('#Bloques', j) for j in urls]
d3=[re.search('/wiki/Act%C3%ADnido', j) for j in urls]
d4=[re.search('/wiki/Lant%C3%A1nido', j) for j in urls]

#Creamos una lista vacía
wiki_url=[]

#Realizamos un bucle donde tomaremos todos los elementos que no contengan algo de las d's
for i in range(len(d1)):
    if d1[i]==None and d2[i]==None and d3[i]==None and d4[i]==None:
        wiki_url.append(urls[i])

#La lista de enlaces de los elementos no aparece ordenada mediante el número atómico 
#lo reordenaremos para que cuadre con nuestros datos
element_url=wiki_url[0:56]+wiki_url[88:103]+wiki_url[56:73]+wiki_url[103:]+wiki_url[73:88]

#Una vez llegados a este punto tendremos todos los enlaces que nos redireccionan hacia el elemento en la página de la 
#wikipedia

#Creamos una lista vacía de elementos donde guardaremos la densidad
densidad=[]

#Realizamos un bucle sobre todas las urls de los elementos
for e in element_url:
    #Entramos en la determinada url de un elemento
    page2 = requests.get('https://es.wikipedia.org' + e ,headers = headers)
    soup2 = BeautifulSoup(page2.text, "lxml")
    #Mostramos todas las etiquetas td que aparecen sobre la página
    t6=soup2.find_all('td')
    #Además extraemos el texto en bruto
    texto6=[d.text for d in t6]
    #Mediante este apartado haremos una búsqueda sobre las unidades de la densidad para encontrar donde se encuentra el valor
    unidad=[re.search('kg/m3',i.text) for i in t6]
   
    #Es ahora cuando haremos uso de la función que nos permite detectar si todos los elementos son iguales
    #Si todos los elementos de la lista unidad son iguales significa que no existe densidad por lo que colocaremos una -
    if todos_elementos_iguales(unidad)==True:
            densidad.append('-')
    #De tener elementos distintos existirá una densidad asociada a esas unidades
    else:
        #Realizamos un bucle sobre la lista unidad para extraer su valor de texto
        for j in range(len(unidad)):
            if unidad[j]!=None:
                #En la lista se insertarán los valores en bruto de la densidad
                densidad.append(texto6[j])

#Por último realizaremos una limpieza sobre los datos de la densidad para obtener los valores en kg/m^3 y eliminar los 
#elementos innecesarios
den=[i.replace("\xa0kg/m3","").replace("\n","").replace("\u200b","").replace("[2]","").replace("[1]","").replace("[3]","")
     .replace('(predicción)',"").replace('(300 K)', "").replace("(26,85\xa0°C)","")
     .replace("(300 K)","").replace("(273 K)","").replace("(a 273 K)","").replace('6,501 g/cm³','6501')
     .replace('Desconocido','-').replace(".","").replace(".",",").replace(" ","") for i in densidad]


#---------------------------------------------------------------------------------------
#Creación del DATASET
#---------------------------------------------------------------------------------------
#Creamos un diccionario donde guardamos todas las variables
tabla_periodica= {"N_atomico": n_atomico , 
                  "Elemento": elemento , 
                  "Nombre": nombre, 
                  "Masa": extraer_masa(masa), 
                  "Periodo":periodo,
                  "Grupo": grupo,
                  "Estructura": estructura,
                  "Conf_electronica":electronic,
                  "Descubrimiento":año(impar),
                  "Densidad":den
                  }

#Creamos el Dataframe a partir del diccionario
tabla=pd.DataFrame(tabla_periodica, columns=Nombre)


#---------------------------------------------------------------------------------------
#Creación del CSV
#---------------------------------------------------------------------------------------
tabla.to_csv('Elementos_tabla_periodica.csv', header=True, index=False)


#---------------------------------------------------------------------------------------
#Breve descripción de los resultados
#---------------------------------------------------------------------------------------
tabla.head(20)

,N_atomico,Elemento,Nombre,Masa,Periodo,Grupo,Estructura,Conf_electronica,Descubrimiento,Densidad
0,1,H,Hidrogeno,"1,007",1,1,Hexagonal,1s1,1766,"0,0899"
1,2,He,Helio,"4,002",1,18,Hexagonal,1s2,1895,"0,1785"
2,3,Li,Litio,"6,941",2,1,Cubica centrada en el cuerpo,[He] 2s1,1817,535
3,4,Be,Berilio,"9,012",2,2,Cubica simple,[He] 2s2,1797,1848
4,5,B,Boro,"10,81",2,13,Romboedrica,[He] 2s2 2p1,1808,2460
